In [3]:
import torch

In [4]:
torch.cuda.is_available()

True

# Tensors

In [ ]:
torch.rand(2, 2)

tensor([[0.9561, 0.2071],
        [0.7359, 0.3666]])

In [ ]:
a = torch.tensor([[1, 0, 0], [0, 1, 0], [0, 0, 1]])
a

tensor([[1, 0, 0],
        [0, 1, 0],
        [0, 0, 1]])

In [ ]:
a[0][0] = 2
a

tensor([[2, 0, 0],
        [0, 1, 0],
        [0, 0, 1]])

In [5]:
torch.zeros(2, 2)

tensor([[0., 0.],
        [0., 0.]])

In [6]:
torch.ones(2, 2)

tensor([[1., 1.],
        [1., 1.]])

In [9]:
a = torch.tensor([[1, 2], [3, 4]])
b = torch.tensor([[5, 6], [7, 8]])
print(a-b)
print(a+b)

tensor([[-4, -4],
        [-4, -4]])
tensor([[ 6,  8],
        [10, 12]])


In [14]:
a.device

device(type='cpu')

In [15]:
a_gpu = a.to('cuda')
a_gpu.device

device(type='cuda', index=0)

Max value of a tensor

In [18]:
a.max().item()

4

Index of the max value of a tensor

In [17]:
a.argmax()

tensor(3)

Changing the type of a tensor

In [20]:
print(a.type())
a = a.to(dtype=torch.float32)
print(a.type())

torch.LongTensor
torch.FloatTensor


in-place functions have the same name as the original function but
with an appended underscore

In [22]:
b = a.log2() #have to store new tensor in a different memory location
a.log2_() #inplace function
a

tensor([[0.0000, 1.0000],
        [1.5850, 2.0000]])

Reshaping a tensor

In [27]:
c = a.reshape(4, 1)
c.shape

#alternatively:
#b = a.view(1, 4)
#but view() operates as a view on the original tensor, and is avoided

torch.Size([4, 1])

Rearrange the dimensions

In [29]:
hwc = torch.rand(640, 480, 3)
chw = hwc.permute(2, 0, 1)
chw.shape

torch.Size([3, 640, 480])

# Data

In [30]:
class Dataset(object):
  def __getitem__(self, index):
    raise NotImplementedError
  
  def __len__(self):
    raise NotImplementedError

In [69]:
import torchvision
from torchvision import transforms

## PyTorch dataset

In [70]:
train_data_path = "/content/drive/MyDrive/Colab Notebooks/Deep Learning/CV/Data/101_ObjectCategories/train"
val_data_path = "/content/drive/MyDrive/Colab Notebooks/Deep Learning/CV/Data/101_ObjectCategories/test"
test_data_path = "/content/drive/MyDrive/Colab Notebooks/Deep Learning/CV/Data/101_ObjectCategories/test"

In [71]:
transforms = transforms.Compose([
                                 transforms.Resize(64), #resizing all images to 64x64 because GPUs perform standard size calculations better
                                 transforms.ToTensor(),
                                 transforms.Normalize(mean=[0.485, 0.456, 0.406], #mean and
                                                      std=[0.229, 0.224, 0.225] #standard deviation of the whole dataset. These values are for the imagenet dataset and work well for other datasets too
                                                      ) #prevents the exploding gradients problem
])

In [72]:
train_data = torchvision.datasets.ImageFolder(
                                              root=train_data_path,
                                              transform=transforms)

val_data = torchvision.datasets.ImageFolder(
                                              root=val_data_path,
                                              transform=transforms)

test_data = torchvision.datasets.ImageFolder(
                                              root=test_data_path,
                                              transform=transforms)

## PyTorch dataloaders

In [73]:
from torch.utils import data

In [74]:
BATCH_SIZE = 64
train_data_loader = data.DataLoader(train_data, batch_size=BATCH_SIZE)
val_data_loader = data.DataLoader(val_data, batch_size=BATCH_SIZE)
test_data_loader = data.DataLoader(test_data, batch_size=BATCH_SIZE)

# Neural Networks

In [75]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [76]:
class Net(nn.Module):
  def __init__(self):
    super(Net, self).__init__() #calling superclass constructor
    self.fc1 = nn.Linear(12288, 84)
    self.fc2 = nn.Linear(84, 50)
    self.fc3 = nn.Linear(50, 4)

  def forward(self):
    x = x.view(-1, 12288)
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = F.softmax(self.fc3(x))
    return x

In [77]:
neuralnet1 = Net()

In [78]:
import torch.optim as optim
optimizer = optim.Adam(neuralnet1.parameters(), lr=0.001)

In [ ]:
if torch.cuda.is_available():
  device = torch.device('cuda')
else:
  device = torch.device('cpu')
model.to(device)

In [ ]:
def train(model, optimizer, loss_fn, train_loader, val_loader, epochs=20, device='cpu'):
  for epoch in range(epochs):
    training_loss = 0.0
    validation_loss = 0.0
    model.train()
    for batch in train_loader:
      optimizer.zero_grad()
      inputs, target = batch
      inputs = inputs.to(device)
      target = targets.to(device)
      output = model(inputs)
      loss = loss_fn(output, target)
      loss.backward()
      optimizer.step()
      training_loss += loss.data.item()
    training_loss /= len(train_iterator)

    model.eval()
    num_correct = 0
    num_examples = 0
    for batch in val_loader:
      inputs, targets = batch
      inputs = inputs.to(device)
      output = model(inputs)
      targets - targets.to(device)
      loss = loss_fn(output, targets)
      valid_loss += loss.data.item()
      correct = torch.eq(torch.max(F.softmax(output), dim=1)[1], target).view(-1)
      num_correct += torch.sum(correct).item()
      num_examples += correct.shape[0]
    valid_loss /= len(valid_iterator)

    print('Epoch: {}, Training Loss: {:.2f},
    Validation Loss: {:.2f},
    accuracy = {:.2f}'.format(epoch, training_loss,
    valid_loss, num_correct / num_examples))


In [ ]:
train(neuralnet1, optimizer, torch.nn.CrossEntropyLoss(),
train_data_loader, test_data_loader,device)